## Préparation du jeu de données PubChem (Ajout des numéros atomiques)

Pour tenter d'améliorer les performances des RN que nous entraînons, nous allons rajouter des informations en entrée. La géométrie des molécules dépendant du nuage électronique, qui dépend entre autres du nombre d'électrons, nous allons ajouter le numéro atomique des atomes en entrée des RN, pour leur éviter d'avoir à créer implicitement une "table de correspondances" entre les masses atomiques déjà fournies et les numéros atomiques.

Dans le même temps, nous allons travailler sur le jeu de données réduit créé dans le notebook 8.0, contenant uniquement les molécules de taille comprise entre 2 et 60, car les données sur les autres molécules sont soit aberrantes pour la tâche que nous effectuons, soit marginales en représentation. Cela va nous permettre de créer des jeux de données d'entrée et de cible pour les RN moins importants et donc de gagner en mémoire et en temps d'exécution.

Pour chaque molécule, nous allons donc créer une entrée du RN contenant pour chaque atome les quatre informations de position, la masse atomique et le numéro atomique. Cela revient donc à 60\*6 nombres soit 360 nombres plutôt que 1000 comme c'était le cas jusqu'ici.

Pour chaque molécule, nous allons également créer un vecteur cible contenant les informations sur les différences de distances attendues sur les quatre informations de position. Cela revient donc à 60\*4 valeurs soit 240 plutôt que 800 comme c'était le cas jusqu'ici.

De plus, nous allons convertir les données concernant les distances en entrée du RN en mÅ, dans l'idée qu'il sera peut être plus simple pour les RN d'effectuer des prédictions correctes si les données d'entrée et de sortie sont dans le même ordre de grandeur.

In [3]:
test_set_location = "../data/test_set_riken_v2_reduced.h5"
test_set_prepared_input_location = "../data/test_set_riken_v2_prepared_input_bruit+_anums_reduced.h2"
test_set_labels_location = "../data/test_set_riken_v2_labels_bruit+_anums_reduced.h2"

train_set_location = "../data/train_set_riken_v2_reduced.h5"
train_set_prepared_input_location = "../data/train_set_riken_v2_prepared_input_bruit+_anums_reduced.h5"
train_set_labels_location = "../data/train_set_riken_v2_labels_bruit+_anums_reduced.h5"

minimal_set_riken_location = "../data/minimal_set_riken_v2_reduced.h5"
minimal_set_prepared_input_location = "../data/minimal_set_riken_v2_prepared_input_bruit+_anums_reduced.h5"
minimal_set_labels_location = "../data/minimal_set_riken_v2_labels_bruit+_anums_reduced.h5"

mini_set_riken_location = "../data/mini_set.h5"
mini_set_prepared_input_location = "../data/mini_set_prepared_input_bruit+_anums_reduced.h5"
mini_set_labels_location = "../data/mini_set_labels_bruit+_anums_reduced.h5"


#### Définition de la fonction d'ajout de bruit

In [4]:
import numpy as np

def positions_bruitees(positions):    
    bruit = np.random.normal(loc=0.0, scale=0.1732, size=positions.shape)
    return ((positions + bruit), bruit)


#### Définition de la fonction de calcul de la matrice de distances aux points du repère fixes compressée à partir de la matrice des coordonnées des atomes

In [5]:
def matrice_distances_compr_abs(positions):
    """ Renvoie la matrice de distances compressée des positions des atomes passées en paramètres
    La matrice de distances compressée est définie de la façon suivante : pour chaque atome, on calcule
    la distance avec chaque point du repère. Une ligne i de la matrice (n,4) correspond aux distances
    de l'atome i avec chacun des quatre points du repère"""
    
    nb_at = len(positions)
    
    # On renvoie un tableau vide si la molécule est vide
    if nb_at == 0:
        return []
    
    repere = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
    repere = np.vstack([repere]*nb_at)

    positions = np.tile(positions, 4).reshape(4*nb_at, 3)
    
    return np.sqrt(np.sum(np.power(positions-repere, 2), 1)).reshape(nb_at, 4)

#### Fonction de création des entrées et des labels des RN


In [6]:
import h5py
import time
import numpy as np

def creation_input_RN(set_location, input_rn_location_rel_abs, labels_location, epochs, nb_mol_mem):
    
    """ nb_mol_mem est le nombre maximal de molécules que l'on stocke simultanément en mémoire. Lorsque la 
    taille est atteinte, on écrit les entrées du RN et les cibles du RN sur le disque (taille max des
    batchs de traitement) 
    La nécessité d'utiliser des batchs de traitement en mémoire est due au fait que le traitement devient très
    long si on ne met pas de données en mémoire et qu'on écrit sur le disque à chaque itération. En utilisant
    les batchs, l'écriture sur le disque se fait en bloc.
    """
    
    start_time = time.time()    
    mol_vides = 0
    
    rmse_epochs = []
    
    print("Creating input and label sets for "+set_location+" : ")

    # On charge le jeu de données original (en lecture seule)
    original_dataset_h5 = h5py.File(set_location, 'r')
    
    # On enregistre la taille du jeu de données
    taille = len(original_dataset_h5["anums"])
    
    # On créé les jeux de données d'entrée du RN et de labels
    input_rn_dataset_h5 = h5py.File(input_rn_location_rel_abs, 'w')
    labels_dataset_h5 = h5py.File(labels_location, 'w')

    # On créé les datasets inputs et targets 
    input_dataset = input_rn_dataset_h5.create_dataset("inputs", shape=(epochs*taille, 360),
                                       dtype=np.float32, compression="gzip", 
                                       chunks=True)

    targets_dataset = labels_dataset_h5.create_dataset("targets", shape=(epochs*taille, 240),
                                       dtype=np.float32, compression="gzip", 
                                       chunks=True)

    
    try:
        
        for k in range(epochs):
            
            print("Epoch "+str(k)+"...")

            # Contient les rmse de tous les bruits ajoutés à chaque molécule durant toute l'époque de génération
            # de l'entrée du RN. À la fin de l'époque, on en fait la moyenne et on l'ajoute à rmse_epochs
            bruits_epoch_rmse = np.empty(shape=(taille,), dtype=np.float32)
           
            # On créé les tableaux np de valeurs du jeu original pour l'époque courante
            input_coords = np.array(original_dataset_h5["riken_coords"])
            input_masses = np.array(original_dataset_h5["amasses"])
            input_nums = np.array(original_dataset_h5["anums"])
            
            # Indice de la première molécule traitée du batch courant (non défni pour le moment)
            indice_batch_courant = None
            
            # On parcourt toutes les molécules
            for i in range(taille):

                # On arrive au début d'un nouveau batch de traitement
                if i%nb_mol_mem == 0:
                    
                    # On écrit les données du batch précédent sur le disque s'il existe un batch précédent
                    if indice_batch_courant is not None :
                        print("Writing data to h5 for epoch "+str(k)+" and indexes from "+
                              str(indice_batch_courant)+" to "+str(indice_batch_courant+taille_batch_courant))
                        
                        input_dataset[k*taille+indice_batch_courant: 
                                      k*taille+indice_batch_courant+taille_batch_courant] = np_input_dataset
                        
                        targets_dataset[k*taille+indice_batch_courant: 
                                            k*taille+indice_batch_courant+taille_batch_courant] = np_targets_dataset
                   
                        
                    # On met à jour la première molécule du batch courant
                    if indice_batch_courant is None :
                        indice_batch_courant = 0
                    else:
                        indice_batch_courant += nb_mol_mem
                    
                    # On calcule la taille du nouveau batch
                    taille_batch_courant = min(nb_mol_mem, taille - i)
                    
                    print("Computing new batch...")
                        
                    # On créé les nouveaux tableaux en mémoire du batch courant
                    np_input_dataset = np.empty(shape=(taille_batch_courant, 360))
                    np_targets_dataset = np.empty(shape=(taille_batch_courant, 240))
                                   
                # On récupère les coordonnées de la molécule courante
                coords = np.array(input_coords[i]).reshape(-1,3)
                
                # On calcule les matrices de distances aux points fixes en mÅ
                dist_init = matrice_distances_compr_abs(coords)*1000
                
                # On insère le bruit sur les coordonnées
                coords_bruit, bruit = positions_bruitees(coords)
                coords_bruit = coords_bruit.reshape(-1, 3)

                # On calcule les différence de distances cibles (en mÅ) et les distances bruitées (en mÅ),
                # pour les distances relatives et absolues
                dist_bruit = matrice_distances_compr_abs(coords_bruit)*1000
                delta_dist_targets = (dist_init - dist_bruit)

                # On récupère les masses atomiques de la molécule courante
                masses = input_masses[i]
                anums = input_nums[i]

                # On initialise l'entrée du RN et le vecteur cible pour la molécule courante
                entree_courante = np.zeros(shape=(360, 1))
                cible_courante = np.zeros(shape=(60, 4))

                # On ajoute les coordonnées bruitées et les masses à l'entrée avec padding, et les coordonnées
                # cibles au dataset targets
                for j in range(len(masses)):

                    # Ajout des distances absolues, des distances relatives et de la masse au vecteur entrée
                    index_input_courant = j*6
                    entree_courante[index_input_courant:index_input_courant+4] = dist_bruit[j].reshape(4,1)
                    entree_courante[index_input_courant+4] = masses[i]
                    entree_courante[index_input_courant+5] = anums[j]

                    # Ajout des données aux matrices cible
                    cible_courante[j] = delta_dist_targets[j]


                # On aplatit les matrices cible
                cible_courante = cible_courante.reshape(1, 240)

                # On insère les données dans le tableau np en mémoire
                np_input_dataset[i%taille_batch_courant] = entree_courante.reshape(1, 360)
                np_targets_dataset[i%taille_batch_courant] = cible_courante

                # On ajoute le rmse ajouté à la molécule au tableau des rmse de l'époque
                np_delta_dist_targets = np.array(delta_dist_targets)
                bruits_epoch_rmse[i] = np.sqrt(np.mean(np.square(np_delta_dist_targets)))
                
            
            
            print("end epoch")
            # On ajoute les données du dernier batch
            print("Writing data to h5 for epoch "+str(k)+" and indexes from "+
                              str(indice_batch_courant)+" to "+str(indice_batch_courant+taille_batch_courant))
                        
            input_dataset[k*taille+indice_batch_courant: 
                          k*taille+indice_batch_courant+taille_batch_courant] = np_input_dataset

            targets_dataset[k*taille+indice_batch_courant: 
                                k*taille+indice_batch_courant+taille_batch_courant] = np_targets_dataset
    
                
            # On ajoute le rmse moyen de l'époque au tableau des rmse de toutes les époques
            rmse_epochs.append(np.mean(bruits_epoch_rmse))
            

        print("Writing datasets to disk")
        input_rn_dataset_h5.flush()
        labels_dataset_h5.flush()

        
        print("--- %s seconds ---" % (time.time() - start_time))
        
        # On calcule le rmse moyen de toutes les époques
        np_rmse_epochs = np.array(rmse_epochs)
        print("RMSE bruit moyen dist : "+str(np.mean(np_rmse_epochs)))

        
    finally:
        original_dataset_h5.close()
        input_rn_dataset_h5.close()
        labels_dataset_h5.close()

    



In [7]:
creation_input_RN(mini_set_riken_location, mini_set_prepared_input_location, mini_set_labels_location, 1,
                 1)

Creating input and label sets for ../data/mini_set.h5 : 
Epoch 0...
Computing new batch...
Writing data to h5 for epoch 0 and indexes from 0 to 1
Computing new batch...
end epoch
Writing data to h5 for epoch 0 and indexes from 1 to 2
Writing datasets to disk
--- 0.0070340633392333984 seconds ---
RMSE bruit moyen dist : 158.24399


In [30]:
creation_input_RN(train_set_location, train_set_prepared_input_location, train_set_labels_location,
                 3, 100000)

Creating input and label sets for ../data/mini_set.h5 : 
Epoch 0...
Computing new batch...
end epoch
Writing data to h5 for epoch 0 and indexes from 0 to 2
Epoch 1...
Computing new batch...
end epoch
Writing data to h5 for epoch 1 and indexes from 0 to 2
Epoch 2...
Computing new batch...
end epoch
Writing data to h5 for epoch 2 and indexes from 0 to 2
Writing datasets to disk
--- 0.009618997573852539 seconds ---
RMSE bruit moyen dist : 175.68146


##### Sortie 

    --- 4717.987053871155 seconds ---
    RMSE bruit moyen dist : 171.76501


In [ ]:
creation_input_RN(test_set_location, test_set_prepared_input_location, train_set_labels_location,
                 3, 100000)

##### Sortie

    --- 502.8285777568817 seconds ---
    RMSE bruit moyen dist : 171.7456
